In [3]:
import pyforest
sys.path.append('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/')
from RF_atomver import *

<IPython.core.display.Javascript object>

# Raw datasets GP performed poorly

In [28]:
df = pd.read_csv('final_paper_models.csv')
df = df[(df['set'] !='literature') & (df['modeling_type']!='RF_GS')]
df = df[~df['set'].isin(['fold1', 'fold2', 'fold3', 'fold4', 'fold5'])]
df['dataset type'] = ['binding' if nek in ['NEK2_binding','NEK3_binding','NEK5_binding','NEK9_binding'] else 'inhibition' for nek in df['NEK']]
df['1-specificity'] = 1-df['specificity'] 
gp = df[(df['modeling_type']=='GP_matern') & (df['strategy']=='raw')]
gp_feat_avg = gp.groupby('feat_type')[['recall', '1-specificity']].mean()
gp_feat_avg

,recall,1-specificity
feat_type,,
mfp,0.005952,0.0
moe,0.000000,0.0


In [29]:
df['set'].value_counts()

set
original            240
batch2_inactives     24
Name: count, dtype: int64

In [30]:
raw_rf = df[(df['RF_type'] == 'RF') | (df['RF_type'] == 'RF_BCW')]
raw_rf = raw_rf[raw_rf['strategy'] =='raw']
rf_raw_avg = raw_rf .groupby('dataset type')[['recall', '1-specificity']].mean()
rf_raw_avg

,recall,1-specificity
dataset type,,
binding,0.191012,0.006642
inhibition,0.595734,0.018884


# all NEK averages 
this means any nek, any feat type, any modeling strategy, just simply average of strategy type 

In [31]:
for samp in df['strategy'].unique():
    samp_df = df[df['strategy']==samp]
    avg = samp_df[['recall', '1-specificity']].mean()
    print(f'{samp} average:\n{avg}')
    print()
    

raw average:
recall           0.427296
1-specificity    0.104096
dtype: float64

UNDER average:
recall           0.640505
1-specificity    0.403364
dtype: float64

SMOTE average:
recall           0.393906
1-specificity    0.028203
dtype: float64

ADASYN average:
recall           0.391709
1-specificity    0.031937
dtype: float64



# Rf averages 

In [51]:
essentially_under = []
# for samp in df['strategy'].unique():
#     samp_df = df[df['strategy']==samp]
#     avg = samp_df[['recall', '1-specificity']].mean()
#     print(f'{samp} average:\n{avg}')
#     print() 
rf= df[df['modeling_type'] =='RF']
rf_raw = rf[(rf['strategy'] == 'raw') & (rf['RF_type'].isin(['RF', 'RF_BCW', 'BRFC', 'BRFC_BCW']))]
rf_other = rf[(rf['RF_type'] == 'RF') & (rf['strategy'].isin(['UNDER','SMOTE']))]  

rf_filtered = pd.concat([rf_raw, rf_other])
total_rf = rf_filtered.copy() 
total_rf['strategy'] =total_rf['strategy'].astype(str) +'_'+ total_rf['RF_type']+'_'+total_rf['set'] 
total_rf['strategy'].unique()

gp= df[df['modeling_type'] == 'GP_matern']
gp['strategy'] = gp['strategy'].astype(str)+'_GP_'+gp['set'] 

full_df = pd.concat([gp, total_rf]) 
# full_df['1-specificity'] = 1-full_df['specificity']
full_df['dataset type'] = full_df['NEK'].str.split('_').str[1]
batch2_inactive_df = full_df[full_df['set'] == 'batch2_inactives']
bind_df2 = full_df[full_df['dataset type'] == 'binding'] 
inhib_df2 = full_df[full_df['dataset type'] == 'inhibition'] 

bind_recalls = []
inhib_recalls = [] 
bind_1specs = []
inhib_1specs=[]
strategies = []
for strat in bind_df2['strategy'].unique(): 
    bind_strat = bind_df2[bind_df2['strategy']==strat] 
    inhib_strat = inhib_df2[inhib_df2['strategy']==strat] 
    avg_bind = bind_strat[['recall', '1-specificity']].mean()
    avg_inhib = inhib_strat[['recall', '1-specificity']].mean()
    print(f'{strat} average:\nbinding:\n{avg_bind}\ninhibition:\n{avg_inhib}')
    
    print()
    bind_recalls.append(avg_bind['recall'])
    bind_1specs.append(avg_bind['1-specificity'])
    inhib_recalls.append(avg_inhib['recall'])
    inhib_1specs.append(avg_inhib['1-specificity'])
    strategies.append(strat) 
    
    
bind_avgs = pd.DataFrame({'strategy': strategies,'recall average': bind_recalls, '1-specificity average': bind_1specs })
bind_avgs['nek dataset'] = 'binding'
inhib_avgs = pd.DataFrame({'strategy': strategies,'recall average': inhib_recalls, '1-specificity average': inhib_1specs })
inhib_avgs['nek dataset'] = 'inhibition'
avgs = pd.concat([bind_avgs, inhib_avgs], axis=1)



raw_GP_original average:
binding:
recall           0.0
1-specificity    0.0
dtype: float64
inhibition:
recall           0.008929
1-specificity    0.000000
dtype: float64

UNDER_GP_original average:
binding:
recall           0.669862
1-specificity    0.433768
dtype: float64
inhibition:
recall           0.642857
1-specificity    0.296117
dtype: float64

SMOTE_GP_original average:
binding:
recall           0.242157
1-specificity    0.007345
dtype: float64
inhibition:
recall           0.582341
1-specificity    0.010332
dtype: float64

ADASYN_GP_original average:
binding:
recall           0.216054
1-specificity    0.007364
dtype: float64
inhibition:
recall           0.610119
1-specificity    0.011648
dtype: float64

UNDER_GP_batch2_inactives average:
binding:
recall           0.715092
1-specificity    0.501584
dtype: float64
inhibition:
recall           0.574405
1-specificity    0.284266
dtype: float64

raw_RF_original average:
binding:
recall           0.177630
1-specificity    0.005862
dt

/var/folders/wb/cvnnx9qn3tj18cq5_9wx39xm0000gn/T/ipykernel_46942/1009876918.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gp['strategy'] = gp['strategy'].astype(str)+'_GP_'+gp['set']


# NEK averages (all models for each NEK) 


In [54]:
neks = [] 

for nek in df['NEK'].unique(): 
    nek_df = df[df['NEK']==nek] 
    nek_avg = nek_df[['recall', '1-specificity']].mean()
    print(f'{nek}\n{nek_avg}\n')

NEK2_binding
recall           0.424242
1-specificity    0.171000
dtype: float64

NEK2_inhibition
recall           0.608766
1-specificity    0.151256
dtype: float64

NEK3_binding
recall           0.425134
1-specificity    0.209262
dtype: float64

NEK5_binding
recall           0.522727
1-specificity    0.160686
dtype: float64

NEK9_binding
recall           0.325175
1-specificity    0.171212
dtype: float64

NEK9_inhibition
recall           0.570707
1-specificity    0.130602
dtype: float64

